In [ ]:
# File: domino.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import numpy as np
from frontend import App

# create an app
app = App.create("domino")

# create a box tetrahedral mesh scaled to domino proportions
V, F, T = app.mesh.box().tetrahedralize().scale(0.1, 0.25, 0.025)
app.asset.add.tet("block", V, F, T)

# create a scene
scene = app.scene.create()

# arrange dominoes in a spiral pattern
R, minR, d, N, C = 1.0, 0.5, 0.15, 4096, 2
blocks, xz = [], None
for i in reversed(range(N)):
    t = 2.0 * C * np.pi * i / N - np.pi / 2
    r = (R - minR) * i / N + minR
    angle = 180 * t / np.pi
    x, z = -r * np.cos(t), r * np.sin(t)
    # only add dominoes with sufficient spacing
    if xz is None or np.linalg.norm(np.array([x, z]) - xz) > d:
        blocks.append(scene.add("block").at(x, 0, z).rotate(angle, "y"))
        xz = np.array([x, z])

# add first domino tilted to start the chain reaction
blocks.append(scene.add("block").at(-0.1, 0.25, -R).rotate(90, "y").rotate(-20, "z"))

# set material properties for all dominoes
gap = 1e-3
for obj in blocks:
    (
        obj.param.set("contact-gap", gap)
        .set("friction", 0.1)
        .set("poiss-rat", 0.49)
    )

# add invisible floor
wall = scene.add.invisible.wall([0, scene.min("y") - 0.5 * gap, 0], [0, 1, 0])

# compile the scene and report stats
scene = scene.build().report()

# set preview options
opts = {"flat_shading": True, "wireframe": True}

# preview the initial scene
scene.preview(options=opts)

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters
(
    session.param.set("min-newton-steps", 32)
    .set("dt", 0.01)
    .set("fps", 30)
    .set("frames", 250)
)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview(options=opts)

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate(options=opts)

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()